In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append('/home/tim/Desktop/Data_science/fastai/')
from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

/home/tim/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Load the bible text

In [2]:
path = './data/'
text = open(path + 'kingjames.txt').read()
text[:300]

'The First Book of Moses:  Called Genesis\n\n\n1:1 In the beginning God created the heaven and the earth.\n\n1:2 And the earth was without form, and void; and darkness was upon\nthe face of the deep. And the Spirit of God moved upon the face of the\nwaters.\n\n1:3 And God said, Let there be light: and there w'

## Get the individual characters

In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print ('Vocab size: {}'.format(vocab_size))

Vocab size: 74


In [4]:
idx2char = {i:j for i, j in enumerate(chars)}
char2idx = {j:i for i,j in enumerate(chars)}

### Convert the text to character indexes which will then be our data

In [5]:
idxs = [char2idx[char] for char in text]

In [6]:
# Sense check that it worked
''.join(idx2char[idxs[i]] for i in range(100))

'The First Book of Moses:  Called Genesis\n\n\n1:1 In the beginning God created the heaven and the earth'

### Let's start with the 3-character model such as was done in lesson 6

In [57]:
# First let's make the 3 letter blocks and the 4 letter as the dependent variable
cs = 3
x_1 = np.array([idxs[i] for i in range(0, len(idxs) - cs, cs)])
x_2 = np.array([idxs[i+1] for i in range(0, len(idxs) - cs, cs)])
x_3 = np.array([idxs[i+2] for i in range(0, len(idxs) - cs, cs)])
y = np.array([idxs[i+4] for i in range(0, len(idxs) - cs, cs)])

Now we can create a model


In [16]:
hidden_layers = 256
embedding_size = int(len(chars) / 2)

class CharModel(nn.Module):
    
    def __init__(self, vocab_size, embedding_size):
        super().__init__()
        
        # First let's make an embedding layer for the characters
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        
        # Then make a hidden layer that does a matrix multiply
        self.hidden_1 = nn.Linear(embedding_size, hidden_layers)
        
        
        # Then make a hidden to hidden layer that does a matrix multiply 
        self.hidden_2 = nn.Linear(hidden_layers, hidden_layers)
        
        # Then make the output layer
        self.out = nn.Linear(hidden_layers, vocab_size)
        
        
    def forward(self, c1, c2, c3):
        
        in_1 = F.relu(self.hidden_1(self.embedding(c1)))
        in_2 = F.relu(self.hidden_1(self.embedding(c2)))
        in_3 = F.relu(self.hidden_1(self.embedding(c3)))
        
        h = V(torch.zeros(in_1.size()))
        
        h = F.tanh(self.hidden_2(in_1 + h))
        h = F.tanh(self.hidden_2(in_2 + h))
        h = F.tanh(self.hidden_2(in_3 + h))
        
        out = F.log_softmax(self.out(h))
        
        return out
        

In [17]:
model = CharModel(vocab_size, embedding_size)
model_data = ColumnarModelData.from_arrays('.', val_idxs=[-1], xs=np.stack([x_1, x_2, x_3],axis=1), y=y, bs=512)

In [18]:
opt = optim.Adam(model.parameters(), lr=1e-2)
fit(model, model_data, 1, opt, crit=F.nll_loss)

epoch      trn_loss   val_loss                                
    0      2.361994   2.800291  



[array([2.80029])]

## Now let's do it by concatenating the h's

In [44]:
# Number of characters
cs=8
c_in_dat = [[idxs[i+j] for i in range(cs)] for j in range(len(idxs)-cs)]
c_out_dat = [idxs[j+cs] for j in range(len(idxs)-cs)]
xs = np.stack(c_in_dat, axis=0)
y = np.stack(c_out_dat)
xs[:cs,:cs]

array([[42, 55, 52,  1, 28, 56, 65, 66],
       [55, 52,  1, 28, 56, 65, 66, 67],
       [52,  1, 28, 56, 65, 66, 67,  1],
       [ 1, 28, 56, 65, 66, 67,  1, 24],
       [28, 56, 65, 66, 67,  1, 24, 62],
       [56, 65, 66, 67,  1, 24, 62, 62],
       [65, 66, 67,  1, 24, 62, 62, 58],
       [66, 67,  1, 24, 62, 62, 58,  1]])

In [46]:
val_idx = get_cv_idxs(len(idxs)-cs-1)
model_data = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

In [58]:
class CharModelConcat(nn.Module):
    
    def __init__(self, vocab_size, embedding_size):
        super().__init__()
        
        # First let's make an embedding layer for the characters
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        
        # Then make a hidden layer that does a matrix multiply
        self.hidden_1 = nn.Linear(embedding_size + hidden_layers, hidden_layers)
        
        
        # Then make a hidden to hidden layer that does a matrix multiply 
        self.hidden_2 = nn.Linear(hidden_layers, hidden_layers)
        
        # Then make the output layer
        self.out = nn.Linear(hidden_layers, vocab_size)
        
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, hidden_layers))
        for c in cs:
            inp = torch.cat((h, self.embedding(c)), 1)
            inp = F.relu(self.hidden_1(inp))
            h = F.tanh(self.hidden_2(inp))
        
        return F.log_softmax(self.out(h), dim=-1)
        

In [59]:
model = CharModelConcat(vocab_size, embedding_size)

In [61]:
opt = optim.Adam(model.parameters(), lr=1e-2)
fit(model=model, data=model_data, n_epochs=1, opt=opt, crit=F.nll_loss)

  1%|          | 81/6770 [00:15<26:17,  4.24it/s, loss=3.3]  

KeyboardInterrupt: 

## Now let's implement it using pytorch's RNN layer

In [99]:
class CharRNN(nn.Module):
    
    def __init__(self, vocab_size, embedding_size):
        super().__init__()
        
        # First let's make an embedding layer for the characters
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        
        # Now let's make an RNN layer
        
        self.rnn = nn.RNN(embedding_size, hidden_layers)
        
        # Then make the output layer
        self.out = nn.Linear(hidden_layers, vocab_size)
        
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, hidden_layers))
        
        inp = self.embedding(torch.stack(cs))
        out, h = self.rnn(inp, h)

        return F.log_softmax(self.out(out[-1]), dim=-1)
        

In [100]:
model = CharRNN(vocab_size, embedding_size)

In [101]:
opt = optim.Adam(model.parameters(), lr=1e-2)
fit(model=model, data=model_data, n_epochs=1, opt=opt, crit=F.nll_loss)

 34%|███▍      | 2288/6770 [06:13<09:25,  7.93it/s, loss=1.72]

KeyboardInterrupt: 